In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd gdrive/MyDrive/Dissertation

/content/gdrive/MyDrive/Dissertation


In [ ]:
%cd DatasetQualityML

/content/gdrive/MyDrive/Dissertation/DatasetQualityML


In [ ]:
import PIL
from PIL import Image
import requests
from io import BytesIO
from PIL import ImageFilter
from PIL import ImageEnhance
from IPython.display import display
import numpy as np
from tqdm import tqdm
from glob import glob
import csv
import os

In [ ]:
train_x = sorted(glob("./data/training/images/*"))
train_y = sorted(glob("./data/training/1st_manual/*"))

test_x = sorted(glob("./data/test/images/*"))
test_y = sorted(glob("./data/test/1st_manual/*"))

print(train_x)
print(train_y)
print(test_x)
print(test_y)
print()
print(len(train_x))
print(len(train_y))
print(len(test_x))
print(len(test_y))

['./data/training/images/21_training.tif', './data/training/images/22_training.tif', './data/training/images/23_training.tif', './data/training/images/24_training.tif', './data/training/images/25_training.tif', './data/training/images/26_training.tif', './data/training/images/27_training.tif', './data/training/images/28_training.tif', './data/training/images/29_training.tif', './data/training/images/30_training.tif', './data/training/images/31_training.tif', './data/training/images/32_training.tif', './data/training/images/33_training.tif', './data/training/images/34_training.tif', './data/training/images/35_training.tif', './data/training/images/36_training.tif', './data/training/images/37_training.tif', './data/training/images/38_training.tif', './data/training/images/39_training.tif', './data/training/images/40_training.tif']
['./data/training/1st_manual/21_manual1.gif', './data/training/1st_manual/22_manual1.gif', './data/training/1st_manual/23_manual1.gif', './data/training/1st_ma

In [ ]:
def getImageIntensity(img, mask):
  intensArr = []
  diffConfArr = []

  for x in range(mask.width):
    for y in range(mask.height):
      if mask.getpixel((x,y)) == 255:
        pixConf = 1 - (img.getpixel((x,y)) / 255)
        neighbourConf = 0
        #get surronding pixels
        neighbours = []
        value = mask.getpixel((x-1,y-1)) if x-1 in range(mask.width) and y-1 in range(mask.height) else -1
        neighbours.append(value) # upper left
        value = mask.getpixel((x-1,y)) if x-1 in range(mask.width) else -1
        neighbours.append(value) # upper middle
        value = mask.getpixel((x-1,y+1)) if x-1 in range(mask.width) and y+1 in range(mask.height) else -1
        neighbours.append(value) # upper right
        value = mask.getpixel((x,y-1)) if y-1 in range(mask.height) else -1
        neighbours.append(value) # left
        value = mask.getpixel((x,y+1)) if y+1 in range(mask.height) else -1
        neighbours.append(value) # right
        value = mask.getpixel((x+1,y-1)) if x+1 in range(mask.width) and y-1 in range(mask.height) else -1
        neighbours.append(value) # lower left
        value = mask.getpixel((x+1,y)) if x+1 in range(mask.width) else -1
        neighbours.append(value) # lower middle
        value = mask.getpixel((x+1,y+1)) if x+1 in range(mask.width) and y+1 in range(mask.height) else -1
        neighbours.append(value) # lower right
        if 0 in neighbours:
          coords = [(x-1,y-1),(x-1,y),(x-1,y+1),(x,y-1),(x,y+1),(x+1,y-1),(x+1,y),(x+1,y+1)]
          intens = []
          for idx, n in enumerate(neighbours):
            if n == 0:
              intens.append(img.getpixel(coords[idx]))
          neighbourAvg = sum(intens) / len(intens)
          neighbourConf = 1 - (neighbourAvg / 255)
          diffConf = pixConf - neighbourConf

          diffConfArr.append(diffConf)
          intensArr.append(neighbourAvg)

  avgIntens = sum(intensArr)/len(intensArr)
  avgDiffConf = sum(diffConfArr)/len(diffConfArr)

  #print(avgIntens)
  #print(avgDiffConf)
  return [avgIntens, avgDiffConf]


In [ ]:
with open("./data/training/trainingIntensity2.csv", 'w', newline='') as f:
  writer = csv.writer(f)
  header = ['image', 'avgIntens', 'avgConf']
  writer.writerow(header)

  for i in range(len(train_x)):
    entry = [os.path.basename(train_x[i])]
    x = Image.open(train_x[i]).convert('L')
    y = Image.open(train_y[i])
    res = getImageIntensity(x, y)
    row = entry + res
    writer.writerow(row)

In [ ]:
with open("./data/test/testIntensity.csv", 'w', newline='') as f:
  writer = csv.writer(f)
  header = ['image', 'avgIntens', 'avgConf']
  writer.writerow(header)

  for i in range(len(test_x)):
    entry = [os.path.basename(test_x[i])]
    x = Image.open(test_x[i]).convert('L')
    y = Image.open(test_y[i])
    res = getImageIntensity(x, y)
    row = entry + res
    writer.writerow(row)